In [3]:
# 本代码由可视化策略环境自动生成 2019年5月8日 16:18
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：每日数据处理函数，每天执行一次
def m19_handle_data_bigquant_run(context, data):
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[
        context.ranker_prediction.date == data.current_dt.strftime('%Y-%m-%d')]
    
    # 当日可用资金
    cash_for_buy = context.portfolio.cash
    
    #　获取当日买入列表,每天选取context.stock_count只
    buy_list = list(ranker_prediction.instrument[:context.stock_count])
    
    #　获取当前持仓
    stock_hold_now = {e.symbol: p.amount * p.last_sale_price
                      for e, p in context.perf_tracker.position_tracker.positions.items()} 

    # 需要卖出的股票:已有持仓中不在买入列表的股票
    stock_to_sell = [ i for i in stock_hold_now if i not in buy_list ]
    stock_to_buy =  [ i for i in buy_list if i not in stock_hold_now ]
    
    # 卖出列表进行卖出操作
    if len(stock_to_sell)>0:
        for instrument in stock_to_sell:
            sid = context.symbol(instrument) # 将标的转化为equity格式
            cur_position = context.portfolio.positions[sid].amount # 持仓
            if cur_position > 0 and data.can_trade(sid):
                context.order_target_percent(sid, 0) # 全部卖出
                # 如果是早盘买早盘卖，卖出的资金可以用于买股票，此时应将下面的注释打开，卖出股票时更新可用现金；
                # 如果是早盘买尾盘卖，则卖出时不需更新可用现金，因为尾盘卖出股票所得现金无法使用
                #cash_for_buy += stock_hold_now[instrument]
    
    # 买入列表执行买操作
    if len(stock_to_buy)>0:
        for instrument,weight in zip(stock_to_buy,context.stock_weights):
            sid = context.symbol(instrument) # 将标的转化为equity格式
            if data.can_trade(sid):
                context.order_target_value(sid, min(cash_for_buy,context.portfolio.portfolio_value*0.5)) # 买入
# 回测引擎：准备数据，只执行一次
def m19_prepare_bigquant_run(context):
    pass

# 回测引擎：初始化函数，只执行一次
def m19_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    context.stock_count = 1
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    #context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, context.stock_count)])
    context.stock_weights = [0.5/context.stock_count for k in range(context.stock_count)]  #半仓买入，每只股票等资金分配
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 0.5
    context.options['hold_days'] = 1



m1 = M.instruments.v2(
    start_date='2010-01-01',
    end_date='2016-01-01',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m2 = M.advanced_auto_labeler.v2(
    instruments=m1.data,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/data_history_data.html
#   添加benchmark_前缀，可使用对应的benchmark数据
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/big_expr.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
shift(close, -2) / shift(open, -1)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用20个分类
all_wbins(label, 20)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)
""",
    start_date='',
    end_date='',
    benchmark='000300.SHA',
    drop_na_label=True,
    cast_label_int=True
)

m3 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
close_0/mean(close_0,5)
close_0/mean(close_0,10)
close_0/mean(close_0,20)
close_0/open_0
open_0/mean(close_0,5)
open_0/mean(close_0,10)
open_0/mean(close_0,20)"""
)

m15 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=0
)

m16 = M.derived_feature_extractor.v3(
    input_data=m15.data,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m7 = M.join.v3(
    data1=m2.data,
    data2=m16.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m13 = M.dropnan.v1(
    input_data=m7.data
)

m6 = M.stock_ranker_train.v5(
    training_ds=m13.data,
    features=m3.data,
    learning_algorithm='排序',
    number_of_leaves=30,
    minimum_docs_per_leaf=1000,
    number_of_trees=20,
    learning_rate=0.1,
    max_bins=1023,
    feature_fraction=1,
    m_lazy_run=False
)

m9 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2016-01-01'),
    end_date=T.live_run_param('trading_date', '2019-01-23'),
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m17 = M.general_feature_extractor.v7(
    instruments=m9.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=0
)

m18 = M.derived_feature_extractor.v3(
    input_data=m17.data,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m14 = M.dropnan.v1(
    input_data=m18.data
)

m8 = M.stock_ranker_predict.v5(
    model=m6.model,
    data=m14.data,
    m_lazy_run=False
)

m19 = M.trade.v4(
    instruments=m9.data,
    options_data=m8.predictions,
    start_date='',
    end_date='',
    handle_data=m19_handle_data_bigquant_run,
    prepare=m19_prepare_bigquant_run,
    initialize=m19_initialize_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)


[2019-05-08 16:00:07.921074] INFO: bigquant: instruments.v2 开始运行..

[2019-05-08 16:00:07.993272] INFO: bigquant: 命中缓存

[2019-05-08 16:00:07.995463] INFO: bigquant: instruments.v2 运行完成[0.074387s].

[2019-05-08 16:00:08.000225] INFO: bigquant: advanced_auto_labeler.v2 开始运行..

[2019-05-08 16:00:08.045582] INFO: bigquant: 命中缓存

...

[2019-05-08 16:07:34.164364] INFO: algo: trading transform...

[2019-05-08 16:07:44.907472] INFO: Performance: Simulated 747 trading days out of 747.

[2019-05-08 16:07:44.909478] INFO: Performance: first open: 2016-01-04 09:30:00+00:00

[2019-05-08 16:07:44.911079] INFO: Performance: last close: 2019-01-23 15:00:00+00:00

[2019-05-08 16:07:49.677193] INFO: bigquant: backtest.v8 运行完成[52.749946s].